In [6]:
import fluidsynth
import magenta
import magenta.music as mm
from magenta.common import merge_hparams
from magenta.contrib import training as contrib_training
from magenta.models.music_vae import data
from magenta.models.music_vae import data_hierarchical
from magenta.models.music_vae import lstm_models
from magenta.models.music_vae.base_model import MusicVAE
from magenta.models.music_vae.trained_model import TrainedModel
from magenta.models.music_vae import configs
import note_seq
import tensorflow
import glob
import collections
import os

In [62]:
## hparams is hyperparameters, increasing free_bits
## and decreasing max_deta creates better replicas but worse random samples
HParams = contrib_training.HParams

## sets up the config class later used to train the model
class Config(collections.namedtuple(
    'Config',
    ['model', 'hparams', 'note_sequence_augmenter',
    'data_converter', 'train_examples_path', 'eval_examples_path',
     'tfds_name'])):
    def values(self):
        return self._asdict()

## sets everything in config to default values
Config.__new__.__defaults__=(None,) * len(Config._fields)

## allows the config class to update itself
def update_config(config, update_dict):
  config_dict = config.values()
  config_dict.update(update_dict)
  return Config(**config_dict)

CONFIG_MAP = {}

In [63]:
midi1_set = glob.glob("./MidiSet1/*.mid")
midi2_set = glob.glob("./MidiSet2/*.mid")
midi3_set = glob.glob("./MidiSet3/*.mid")
note1_set = []
note2_set = []
note3_set = []
# imports 3 sets of songs from the files in the project
# converts these midi files to note sequences and stores in a list
def import_songs():
    from note_seq.protobuf import music_pb2
    count = 0

    for x in midi1_set:
        sequence = note_seq.midi_file_to_note_sequence(midi1_set[count])
        note1_set.append(sequence)
        # note_seq.play_sequence(sequence, synth=note_seq.synthesize)
        count += 1
    count = 0
    for x in midi2_set:
        sequence = note_seq.midi_file_to_note_sequence(midi2_set[count])
        note2_set.append(sequence)
        # note_seq.play_sequence(sequence, synth=note_seq.synthesize)
        count += 1
    count = 0
    for x in midi3_set:
        sequence = note_seq.midi_file_to_note_sequence(midi3_set[count])
        note3_set.append(sequence)
        # note_seq.play_sequence(sequence, synth=note_seq.synthesize)
        count += 1
import_songs()

d:\pycharmprojects\musicgenerator\venv\lib\site-packages\pretty_midi\pretty_midi.py:101: RuntimeWarning: Tempo, Key or Time signature change events found on non-zero tracks.  This is not a valid type 0 or type 1 MIDI file.  Tempo, Key or Time Signature may be wrong.
  RuntimeWarning)


In [64]:
## initializes a 16 bar trio model to train
trio_16bar_converter = data.TrioConverter(
    steps_per_quarter=4,
    slice_bars=16,
    gap_bars=2)
## simplest version of the 16 bar trio because it has the best generation
## while still being something I understand
CONFIG_MAP['hierdec-trio_16bar'] = Config(
    model=MusicVAE(
        lstm_models.BidirectionalLstmEncoder(),
        lstm_models.HierarchicalLstmDecoder(
            lstm_models.SplitMultiOutLstmDecoder(
                core_decoders=[
                    lstm_models.CategoricalLstmDecoder(),
                    lstm_models.CategoricalLstmDecoder(),
                    lstm_models.CategoricalLstmDecoder()],
                output_depths=[
                    90,  # melody
                    90,  # bass
                    512,  # drums
                ]),
            level_lengths=[16, 16],
            disable_autoregression=True)),
    hparams=merge_hparams(
        lstm_models.get_default_hparams(),
        HParams(
            batch_size=256,
            max_seq_len=256,
            z_size=512,
            enc_rnn_size=[2048, 2048],
            dec_rnn_size=[1024, 1024],
            free_bits=256,
            max_beta=0.2,
        )),
    note_sequence_augmenter=None,
    data_converter=trio_16bar_converter,
    train_examples_path=None,
    eval_examples_path=None,
)

In [7]:
def interpolate(model, start_seq, end_seq, num_steps, max_length=32,
                assert_same_length=True, temperature=0.5,
                individual_duration=4.0):
  """Interpolates between a start and end sequence."""
  note_sequences = model.interpolate(
      start_seq, end_seq,num_steps=num_steps, length=max_length,
      temperature=temperature,
      assert_same_length=assert_same_length)

  print('Interpolation')
  interp_seq = mm.sequences_lib.concatenate_sequences(
      note_sequences, [individual_duration] * len(note_sequences))

  return interp_seq if num_steps > 3 else note_sequences[num_steps // 2]

def download(note_sequence, filename):
  mm.sequence_proto_to_midi_file(note_sequence, filename)
  download(filename)

print('Done')
#hierdec_trio_16bar_config = CONFIG_MAP['hierdec-trio_16bar']

#music_vae = TrainedModel(
    #configs.CONFIG_MAP['hierdec-trio_16bar'],
   # batch_size= 4,
    #checkpoint_dir_or_path='/checkpoints/hierdec-trio_16bar.ckpt'
#)

#music_vae_train \
#--config=flat-trio_16bar \
#--run_dir=/tmp/music_vae/ \
#--mode=train \
#--examples_path = MidiSet1

INFO:tensorflow:Building MusicVAE model with BidirectionalLstmEncoder, HierarchicalLstmDecoder, and hparams:
{'max_seq_len': 256, 'z_size': 512, 'free_bits': 256, 'max_beta': 0.2, 'beta_rate': 0.0, 'batch_size': 4, 'grad_clip': 1.0, 'clip_mode': 'global_norm', 'grad_norm_clip_to_zero': 10000, 'learning_rate': 0.001, 'decay_rate': 0.9999, 'min_learning_rate': 1e-05, 'conditional': True, 'dec_rnn_size': [1024, 1024], 'enc_rnn_size': [2048, 2048], 'dropout_keep_prob': 1.0, 'sampling_schedule': 'constant', 'sampling_rate': 0.0, 'use_cudnn': False, 'residual_encoder': False, 'residual_decoder': False, 'control_preprocessing_rnn_size': [256]}
INFO:tensorflow:
Encoder Cells (bidirectional):
  units: [2048, 2048]

INFO:tensorflow:
Hierarchical Decoder:
  input length: 256
  level output lengths: [16, 16]

INFO:tensorflow:
Decoder Cells:
  units: [1024, 1024]

INFO:tensorflow:
Decoder Cells:
  units: [1024, 1024]

INFO:tensorflow:
Decoder Cells:
  units: [1024, 1024]

Instructions for updating:

d:\pycharmprojects\musicgenerator\venv\lib\site-packages\tensorflow\python\keras\legacy_tf_layers\core.py:171: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  warnings.warn('`tf.layers.dense` is deprecated and '
d:\pycharmprojects\musicgenerator\venv\lib\site-packages\tensorflow\python\keras\engine\base_layer_v1.py:1719: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '
d:\pycharmprojects\musicgenerator\venv\lib\site-packages\tensorflow\python\keras\engine\base_layer_v1.py:1727: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


ValueError: The passed save_path is not a valid checkpoint: /checkpoints/hierdec-trio_16bar.ckpt

In [12]:
##Generates samples based off the trained model's checkpoint
config = 'hierdec-mel_16bar'
checkpoint = 'hierdec-mel_16bar.tar'
mode = 'sample'
num_output = 5
output = 'generated'

cmd = f'music_vae_generate --config={config} --checkpoint_file={checkpoint} ' \
      f'--mode={mode} --num_outputs={num_output} --output_dir={output}'

print('Running...')
print(os.system(cmd))
print('Done')


Running...
0
Done


In [15]:
##Interpolates between provided samples
mode = 'interpolate'
num_output = 5
inone = 'MidiSet2/Test1.mid'
intwo = 'MidiSet2/Test2.mid'

cmd = f'music_vae_generate --config{config} --checkpoint_file={checkpoint}' \
    f'--mode={mode} --num_outputs={num_output} --input_midi_1={inone} --input_midi_2={intwo}' \
    f'--output_dir={output}'

#music_vae_generate --config=hierdec-mel_16bar --checkpoint_file=hierdec-mel_16bar.tar --mode=interpolate --num_outputs=1 --input_midi_1=MidiSet2/Test1.mid --input_midi_2=MidiSet2/Test2.mid --output_dir=generated

print('Running...')
print(os.system(cmd))
print('Done')
#interp_seq=note_seq.sequences_lib.concatenate_sequences(note_seq)
#note_seq.play_sequence(interp_seq, synth=note_seq.fluidsynth)

Running...
1
Done
